In [1]:
import numpy as np
import networkx as nx
import collections
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [2]:
import imageio
import tqdm
from os.path import isfile, join
from IPython.utils import io
import gc
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

In [3]:
import time

In [4]:
# constants
FLOAT_PRECISION = 16

In [5]:
# set mpl rcParams
mpl.rcParams['font.sans-serif'] = 'Computer Modern Sans Serif'
mpl.rcParams['font.family'] = 'sans-serif'
# mpl.rc('text.latex', preamble=r'\\usepackage[russian]{babel}')
# mpl.rcParams['text.usetex'] = True

mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['xtick.labelsize'] = 24
mpl.rcParams['ytick.labelsize'] = 24
mpl.rcParams['xtick.major.size'] = 14
mpl.rcParams['ytick.major.size'] = 14
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.width'] = 2

mpl.rcParams['axes.edgecolor'] = "white"

mpl.rcParams['xtick.color'] = "white"
mpl.rcParams['ytick.color'] = "white"
mpl.rcParams['xtick.labelcolor'] = "black"
mpl.rcParams['ytick.labelcolor'] = "black"

mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['axes.labelsize'] = 30

In [6]:
def network_portrait(G, trim_lengths=True, trim_numbers=False):
    lengths = dict(nx.all_pairs_shortest_path_length(G))

    # B_{ℓ,k} ≡ the number of nodes who have k nodes at distance ℓ
    n_nodes = G.number_of_nodes()
    res = np.zeros((n_nodes, n_nodes), dtype=np.int32)

    for key, data in lengths.items():
        counters = collections.Counter(data.values())
        for dist in counters:
            if dist == 0:
                continue
            res[dist, counters[dist]] += 1

    if trim_lengths:
        res = res[~np.all(res == 0, axis=1)]

    if trim_numbers:
        emptys = np.all(res == 0, axis=0)
        i = 0
        for e in emptys[::-1]:
            if not e:
                break
            i += 1

        res = res[:, :-i]

    return res

In [92]:
def network_portrait_v2(G):
    lengths = dict(nx.all_pairs_shortest_path_length(G))

    # B_{ℓ,k} ≡ the number of nodes who have k nodes at distance ℓ
    n_nodes = G.number_of_nodes()
    res = np.zeros((n_nodes, n_nodes), dtype=np.int32)

    for key, data in lengths.items():
        counters = collections.Counter(data.values())
        for dist in counters:
            if dist == 0:
                continue
            res[dist, counters[dist]] += 1
    
    res = res[~np.all(res == 0, axis=1)]
    
    for i in range(len(res)):
        res[i, 0] = n_nodes - res[i].sum()

    return res

In [7]:
def to_same_shape(portraits):
    # just adds zeroed rows
    l_max = max([len(p) for p in portraits])
    k_max = portraits[0].shape[1]  # supposedly equals to G.number_of_nodes()
    # implicitly assumes that k_max is the same for all portraits
    # which is true by construction (see 'network_portraits') if
    # the number of nodes is the same
    for i in range(len(portraits)):
        n_rows_to_insert = l_max - len(portraits[i])
        if n_rows_to_insert > 0:
            portraits[i] = np.vstack((portraits[i], np.zeros((n_rows_to_insert, k_max), dtype=np.int32)))
    return portraits

In [113]:
def to_same_shape_v2(portraits):
    l_max = max([p.shape[0] for p in portraits])
    k_max = portraits[0].shape[1]  # implicitly assumes that (1) k_max == N and (2) k_max is the same for all portraits
    for i in range(len(portraits)):
        n_rows_to_insert = l_max - len(portraits[i])
        if n_rows_to_insert > 0:
            arr_to_append = np.zeros((n_rows_to_insert, k_max), dtype=np.int32)
            for j in range(len(arr_to_append)):
                arr_to_append[j, 0] = k_max
            portraits[i] = np.vstack((portraits[i], arr_to_append))
    return portraits

In [56]:
def portrait_fig(port, color_log_scale=True):
    fig, ax = plt.subplots(figsize=(40, 12))
    ax.set_ylabel("distance l")
    ax.set_xlabel("neighbours k")
    ax.invert_yaxis()

    port_ = port + 1
    vmin=port_.min()
    vmax=port_.max()
    if color_log_scale:
        norm = colors.LogNorm(vmin=vmin, vmax=vmax)
    else:
        norm = colors.Normalize(vmin=vmin, vmax=vmax)        
    im = ax.pcolormesh(port_, norm=norm)
    fig.colorbar(im, ax=ax)
    return fig, ax

In [8]:
def portrait_figs(portraits, labels, same_shape=True, color_log_scale=False, color_abs_scale=False):
    # TODO: log scale
    if same_shape:
        portraits = to_same_shape(portraits)

    figs, axes = [], []
    
    if color_abs_scale:
        vmin = min([p.min() for p in portraits]) + 1
        vmax = max([p.max() for p in portraits]) + 1
    
    for i, port in enumerate(portraits):
        fig, ax = plt.subplots(figsize=(40, 12))
        ax.set_title(labels[i])
        ax.set_ylabel("distance l")
        ax.set_xlabel("neighbours k")
        ax.invert_yaxis()

        port_ = port + 1
        if not color_abs_scale:
            vmin=port_.min()
            vmax=port_.max()
        if color_log_scale:
            norm = colors.LogNorm(vmin=vmin, vmax=vmax)
        else:
            norm = colors.Normalize(vmin=vmin, vmax=vmax)        
        im = ax.pcolormesh(port_, norm=norm)
        fig.colorbar(im, ax=ax)
        
        figs.append(fig)
        axes.append(ax)
        print(f"Figure {i+1} with label '{labels[i]}' completed.")
#         plt.savefig(f"./media/tmp_{int(time.time())}_{labels[i]}.png", dpi=300)  # ; exit()
        plt.close(fig)

    return figs

In [9]:
def avg_portrait(portraits):
    return np.sum(portraits, axis=0) / len(portraits)

In [10]:
def er_portrait(n, p, n_ensemble=None):
    if not((n_ensemble is None) or (n_ensemble <= 1)):
        graphs_ensemble = [nx.erdos_renyi_graph(n, p, seed=None, directed=False) for _ in range(n_ensemble)]
        portraits_ensemble = to_same_shape([network_portrait(g) for g in graphs_ensemble])
        return avg_portrait(portraits_ensemble)
    else:
        return network_portrait(nx.erdos_renyi_graph(n, p, seed=None, directed=False))

In [11]:
def er_portraits(n, p_range, n_ensemble=None, p_precision=None):
    portraits = []
    labels = []
    fmt_str = f".{p_precision}f" if (p_precision is not None) else ""
    for p in p_range:
        portraits.append(er_portrait(n, p, n_ensemble=n_ensemble))
        labels.append(f"ER N={n}, p={round(p, FLOAT_PRECISION):{fmt_str}}")
        print(labels[-1])
    return portraits, labels

In [12]:
def rrg_portrait(n, d, n_ensemble=None):
    if not((n_ensemble is None) or (n_ensemble <= 1)):
        graphs_ensemble = [nx.random_regular_graph(d, n, seed=None) for _ in range(n_ensemble)]
        portraits_ensemble = to_same_shape([network_portrait(g) for g in graphs_ensemble])
        return avg_portrait(portraits_ensemble)
    else:
        return network_portrait(nx.random_regular_graph(d, n, seed=None))

In [13]:
def rrg_portraits(n, d_range, n_ensemble=None):
    portraits = []
    labels = []
    for d in d_range:
        portraits.append(rrg_portrait(n, d, n_ensemble=n_ensemble))
        labels.append(f"RRG N={n}, d={d}")
        print(labels[-1])
    return portraits, labels

In [14]:
def ba_portrait(n, m, n_ensemble=None):
    if not((n_ensemble is None) or (n_ensemble <= 1)):
        graphs_ensemble = [nx.barabasi_albert_graph(n, m, seed=None, initial_graph=None) for _ in range(n_ensemble)]
        portraits_ensemble = to_same_shape([network_portrait(g) for g in graphs_ensemble])
        return avg_portrait(portraits_ensemble)
    else:
        return network_portrait(nx.barabasi_albert_graph(n, m, seed=None, initial_graph=None))

In [15]:
def ba_portraits(n, m_range, n_ensemble):
    portraits = []
    labels = []
    for m in m_range:
        portraits.append(ba_portrait(n, m, n_ensemble=n_ensemble))
        labels.append(f"BA N={n}, m={m}")
        print(labels[-1])
    return portraits, labels

In [16]:
def turn_figure_to_image(fig):
    canvas = FigureCanvas(fig)
    ax = fig.gca()
    canvas.draw()  # draw the canvas, cache the renderer

    width, height = fig.get_size_inches() * fig.get_dpi()
    image = np.fromstring(canvas.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)
    plt.close(fig)

    return image

In [17]:
def create_gif(figlist, name, fps=1):
    images = []
    for fig in tqdm.tqdm(figlist, position=0, leave=True):
        images.append(turn_figure_to_image(fig))
        del fig

    gc.collect()

    imageio.mimsave(join("./", name + '.mkv'), images, fps=fps)

In [18]:
def er(N=100, p_start=0.001, p_end=0.101, p_step=0.001, fps=5, n_ensemble=100, color_log_scale=True, color_abs_scale=True):
    p_range = np.arange(p_start, p_end, p_step)
    gen_ensemble = not((n_ensemble is None) or (n_ensemble <= 1))
    portraits, labels = er_portraits(n=N, p_range=p_range, n_ensemble=n_ensemble, p_precision=3)
    figlist = portrait_figs(portraits=portraits, labels=labels, same_shape=True,
                            color_log_scale=color_log_scale, color_abs_scale=color_abs_scale)
    gif_path = f"./media/er_" + gen_ensemble * f"ensemble{n_ensemble}_" + f"n{N}_p{p_start}_{p_end}_fps{fps}"
    create_gif(figlist, gif_path, fps=fps)

In [19]:
def rrg(N=500, d_start=3, d_end=31, fps=3, n_ensemble=100, color_log_scale=True, color_abs_scale=True):
    d_range = np.arange(d_start, d_end)
    gen_ensemble = not((n_ensemble is None) or (n_ensemble <= 1))
    portraits, labels = rrg_portraits(n=N, d_range=d_range, n_ensemble=n_ensemble)
    figlist = portrait_figs(portraits=portraits, labels=labels, same_shape=True,
                            color_log_scale=color_log_scale, color_abs_scale=color_abs_scale)
    gif_path = f"./media/rrg_" + gen_ensemble * f"ensemble{n_ensemble}_" + f"n{N}_d{d_start}_{d_end}_fps{fps}"
    create_gif(figlist, gif_path, fps=fps)

In [20]:
def ba(N=500, m_start=3, m_end=31, fps=3, n_ensemble=None, color_log_scale=True, color_abs_scale=True):
    m_range = np.arange(m_start, m_end)
    gen_ensemble = not((n_ensemble is None) or (n_ensemble <= 1))
    portraits, labels = ba_portraits(n=N, m_range=m_range, n_ensemble=n_ensemble)
    figlist = portrait_figs(portraits=portraits, labels=labels, same_shape=True,
                            color_log_scale=color_log_scale, color_abs_scale=color_abs_scale)
    gif_path = f"./media/ba_" + gen_ensemble * f"ensemble{n_ensemble}_" + f"n{N}_m{m_start}_{m_end}_fps{fps}"
    create_gif(figlist, gif_path, fps=fps)

In [28]:
if __name__ == '__main__':
    er(N=100, p_start=0.001, p_end=0.101, p_step=0.001, fps=5, n_ensemble=100, color_log_scale=True, color_abs_scale=False)
#     rrg(N=500, d_start=3, d_end=26, fps=3, n_ensemble=100, color_log_scale=True, color_abs_scale=True)
#     ba(N=1000, m_start=3, m_end=26, fps=3, n_ensemble=3, color_log_scale=True, color_abs_scale=True)

ER N=100, p=0.001
ER N=100, p=0.002
ER N=100, p=0.003
ER N=100, p=0.004
ER N=100, p=0.005
ER N=100, p=0.006
ER N=100, p=0.007
ER N=100, p=0.008
ER N=100, p=0.009
ER N=100, p=0.010
ER N=100, p=0.011
ER N=100, p=0.012
ER N=100, p=0.013
ER N=100, p=0.014
ER N=100, p=0.015
ER N=100, p=0.016
ER N=100, p=0.017
ER N=100, p=0.018
ER N=100, p=0.019
ER N=100, p=0.020
ER N=100, p=0.021
ER N=100, p=0.022
ER N=100, p=0.023
ER N=100, p=0.024
ER N=100, p=0.025
ER N=100, p=0.026
ER N=100, p=0.027
ER N=100, p=0.028
ER N=100, p=0.029
ER N=100, p=0.030
ER N=100, p=0.031
ER N=100, p=0.032
ER N=100, p=0.033
ER N=100, p=0.034
ER N=100, p=0.035
ER N=100, p=0.036
ER N=100, p=0.037
ER N=100, p=0.038
ER N=100, p=0.039
ER N=100, p=0.040
ER N=100, p=0.041
ER N=100, p=0.042
ER N=100, p=0.043
ER N=100, p=0.044
ER N=100, p=0.045
ER N=100, p=0.046
ER N=100, p=0.047
ER N=100, p=0.048
ER N=100, p=0.049
ER N=100, p=0.050
ER N=100, p=0.051
ER N=100, p=0.052
ER N=100, p=0.053
ER N=100, p=0.054
ER N=100, p=0.055
ER N=100, 

  0%|                                                  | 0/100 [00:00<?, ?it/s]C:\Users\Yuri\AppData\Local\Temp/ipykernel_10168/3724362315.py:7: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  image = np.fromstring(canvas.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)
100%|████████████████████████████████████████| 100/100 [00:29<00:00,  3.42it/s]


In [66]:
def c_lk(portrait):
    C = np.zeros(portrait.shape)
    for l in range(len(portrait)):
        row_sum = sum(portrait[l])
        for k in range(len(portrait[l])):
            C[l, k] = portrait[l, :k].sum() / row_sum
    return C

In [125]:
def p_lk(portrait):
    P = np.zeros(portrait.shape)
    for l in range(len(portrait)):
        row_sum = sum(portrait[l])
        for k in range(len(portrait[l])):
            P[l, k] = portrait[l, k]  / row_sum
    return P

In [155]:
def p_l(portrait):
    n_rows, N = portrait.shape
    N_sq = N**2
    P = np.zeros(n_rows)
    for l in range(n_rows):
        P[l] = sum(k * portrait[l, k] for k in range(N)) / N_sq
    return P

In [134]:
def ks_stat(C1, C2):
    assert len(C1) == len(C2), "Diameters of cumulative distribution arrays are not equal."
    C_delta = np.abs(np.subtract(C1, C2))
    n_rows = len(C_delta)
    K = np.zeros(n_rows)
    for l in range(n_rows):
        K[l] = max(C_delta[l])
    return K

In [140]:
def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

In [152]:
er_port_1 = network_portrait_v2(nx.erdos_renyi_graph(n=500, p=0.005, seed=None))
er_port_2 = network_portrait_v2(nx.erdos_renyi_graph(n=500, p=0.005, seed=None))

er_port_1, er_port_2 = to_same_shape_v2([er_port_1, er_port_2])


P_1 = p_l(er_port_1)
P_2 = p_l(er_port_2)

# C_1 = c_lk(er_port_1)
# C_2 = c_lk(er_port_2)

# P_1 = p_lk(er_port_1)
# P_2 = p_lk(er_port_2)

# for row in P_2:
#     print(sum(row))

# plt.plot(list(range(len(P_1[3]))), P_1[3])

# K_er = ks_stat(C_1, C_2)
# K_er
# plt.plot(K_er)

# portrait_fig(er_port_2)

In [154]:
sum(P_1)

0.819276